In [1]:
import ipywidgets
from pathlib import Path

In [2]:
# config 
bar_width = 150
title_height = 100
favorite_folders = ["/Users/niwn/ipywidget-ws/","/Users/niwn/Downloads/"]

In [3]:
ipywidgets.Select(
    options=["a","b","c"]
)

Select(options=('a', 'b', 'c'), value='a')

In [4]:
# Utils widgets
from ipywidgets import Button, Layout, jslink, IntText, IntSlider

def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

top_left_button = create_expanded_button("Top left", 'info')
top_right_button = create_expanded_button("Top right", 'success')
bottom_left_button = create_expanded_button("Bottom left", 'danger')
bottom_right_button = create_expanded_button("Bottom right", 'warning')

top_left_text = IntText(description='Top left', layout=Layout(width='auto', height='auto'))
top_right_text = IntText(description='Top right', layout=Layout(width='auto', height='auto'))
bottom_left_slider = IntSlider(description='Bottom left', layout=Layout(width='auto', height='auto'))
bottom_right_slider = IntSlider(description='Bottom right', layout=Layout(width='auto', height='auto'))

In [5]:
from ipywidgets import GridspecLayout

grid = GridspecLayout(4, 3)

for i in range(4):
    for j in range(3):
        grid[i, j] = create_expanded_button('Button {} - {}'.format(i, j), 'warning')
grid

GridspecLayout(children=(Button(button_style='warning', description='Button 0 - 0', layout=Layout(grid_area='w…

In [6]:
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

In [7]:
from ipywidgets import Layout, Button, Box, HBox, VBox, Select
from typing import Union

In [8]:
from ipywidgets import Layout, Button, VBox, Label

In [9]:
from traitlets.utils.bunch import Bunch
from ipywidgets import Output
from IPython.display import display

In [10]:
def generate_selector(folder_path:Union[Path,str], callback = lambda :None,  **kwargs) -> Select:
    """"""
    options = [
        path.stem + ("/" if path.is_dir() else "")
        for path in 
        Path(folder_path).iterdir()
    ]
    select = Select(
        options = options,
        **kwargs
    )
    select.observe(callback)
    return select
generate_selector(".")

Select(options=('Untitled1', 'make-gui', 'README', 'Pipfile', '.git/', '.vscode/', 'Pipfile'), value='Untitled…

In [14]:
from pprint import pprint

In [19]:
class SelectHandler():
    pass

class Explorer(HBox):
    def __init__(self):
        sidebar = VBox(
            [create_expanded_button(Path(folder_path).stem+"/","info") for folder_path in favorite_folders],
            layout=Layout(width="150px", display = 'flex', align_items = 'stretch',flex_flow = 'column'),
            height="100%")
        # title_box = Button(description="title_box", button_style="warning", layout=Layout(height='80px', width='auto'))
        title_box = Output(layout=Layout(border= '1px solid black',height="100px"))
        print(".")

        @title_box.capture()
        def on_value_change(change: Bunch) -> None:
            # 値が変更されたイベントを扱う
            # label -> value -> indexの順番に変化する
            if change['name'] == 'value':
                title_box.clear_output()
                # 変更前と変更後の値を出力する
                old_value = change['old']
                new_value = change['new']
                print(f'value changed: {old_value} -> {new_value}')
                pprint(f'value changed: {change}')

        # folder explorerの部分
        item_layout = Layout(min_width="150px",width="150px")
        items = [generate_selector(".", callback = on_value_change, rows=12, layout=item_layout) for i in range(15)]
        box_layout = Layout(overflow='scroll hidden',
            width='auto',
            height='100%',
            flex_flow='row',
            display='flex')
        carousel = Box(children=items, layout=box_layout)
        folder_content = VBox([carousel],layout=Layout(height='100%', width='100%'))

        right_box = VBox([title_box,folder_content],layout=Layout(width="100%", display = 'flex', align_items = 'stretch',flex_flow = 'column',height="100%"))

        super(HBox, self).__init__([sidebar,right_box],layout=Layout(width="100%", display = 'flex', align_items = 'stretch',height="300px"))

    def update_carousel(self):
        pass

explorer = Explorer()
explorer

.


Explorer(children=(VBox(children=(Button(button_style='info', description='ipywidget-ws/', layout=Layout(heigh…